## Продвинутое машинное обучение: 
## Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.
В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:
https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

### 1 часть
Реализуйте базовый частотный метод по Шерлоку Холмсу:

* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.


In [1]:
# Загружаем библиотеки
import numpy as np
from nltk import ngrams
from collections import Counter
import random
import tqdm

In [2]:
# Загрузим текст из двух произведений

text = ''
for file_name in ['data/AnnaKarenina.txt', 'data/WarAndPeace.txt']: 
    with open(file_name, 'r', encoding="utf-8") as f:
        text += f.read()

In [3]:
# Очистим текст, уберем пунктуацию, оставим только русские буквы и пробелы, приведем все к нижнему регистру
ru_alpha = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя "


def clean_text(text):
    text = text.lower()
    text = ''.join([x for x in text if x in ru_alpha])
    return text


text = clean_text(text)

In [122]:
# Подсчитаем частотность букв в тексте
def get_freq_ngrams(text, n=1):
    c = Counter(ngrams(text, n=n)).most_common()
    freq_ngrams = {k[0]: v for k, v in dict(c).items()}
    return freq_ngrams

freq_ngrams = get_freq_ngrams(text, n=1)
freq_ngrams

{' ': 397267,
 'о': 223691,
 'е': 166169,
 'а': 162313,
 'н': 133258,
 'и': 129712,
 'т': 115258,
 'с': 103252,
 'л': 98191,
 'в': 91386,
 'р': 80859,
 'к': 67788,
 'д': 58019,
 'м': 56467,
 'у': 53583,
 'п': 47940,
 'я': 42921,
 'ь': 38351,
 'г': 36870,
 'ы': 36447,
 'б': 34028,
 'з': 32723,
 'ч': 31207,
 'ж': 21480,
 'й': 21072,
 'ш': 17158,
 'х': 15586,
 'ю': 12307,
 'э': 6648,
 'ц': 6172,
 'щ': 5568,
 'ф': 2990,
 'ъ': 695,
 'ё': 462}

In [123]:
# Создадим маппинг для кодирования текста
def encode_mapping(freq_ngrams):
    x = random.sample(freq_ngrams.keys(), k=len(freq_ngrams.keys()))

    assert len(x) == len(freq_ngrams.keys())
    
    enc_mapping = dict(zip(freq_ngrams.keys(), x))
    
    return enc_mapping


enc_mapping = encode_mapping(freq_ngrams)
enc_mapping

{' ': 'в',
 'о': 'х',
 'е': 'а',
 'а': ' ',
 'н': 'к',
 'и': 'н',
 'т': 'ё',
 'с': 'д',
 'л': 'ъ',
 'в': 'ц',
 'р': 'ф',
 'к': 'е',
 'д': 'ж',
 'м': 'л',
 'у': 'э',
 'п': 'ы',
 'я': 'г',
 'ь': 'б',
 'г': 'ю',
 'ы': 'о',
 'б': 'у',
 'з': 'т',
 'ч': 'з',
 'ж': 'я',
 'й': 'п',
 'ш': 'ч',
 'х': 'й',
 'ю': 'щ',
 'э': 'м',
 'ц': 'ь',
 'щ': 'с',
 'ф': 'и',
 'ъ': 'ш',
 'ё': 'р'}

In [124]:
test_text = "В это время Левин часто бывал в доме Щербацких и влюбился в дом Щербацких. Как это ни странно может показаться, но Константин Левин был влюблен именно в дом, в семью, в особенности в женскую половину семьи Щербацких. Сам Левин не помнил своей матери, и единственная сестра его была старше его, так что в доме Щербацких он в первый раз увидал ту самую среду старого дворянского, образованного и честного семейства, которой он был лишен смертью отца и матери. Все члены этой семьи, в особенности женская половина, представлялись ему покрытыми какою-то таинственною, поэтическою завесой, и он не только не видел в них никаких недостатков, но под этой поэтическою, покрывавшею их завесой предполагал самые возвышенные чувства и всевозможные совершенства. Для чего этим трем барышням и нужно было говорить через день по-французски и по-английски; для чего они в известные часы играли попеременкам на фортепиано, звуки которого всегда слышались у брата наверху, где занимались студенты; для чего ездили эти учителя французской литературы, музыки, рисованья, танцев; для чего в известные часы все три барышни с m-lle Linon подъезжали в коляске к Тверскому бульвару в своих атласных шубках – Долли в длинной, Натали в полудлинной, а Кити совершенно в короткой, так что статные ножки ее в туго натянутых красных чулках были на всем виду; для чего им, в сопровождении лакея с золотою кокардой на шляпе, нужно было ходить по Тверскому бульвару, – всего этого и многого другого, что делалось в их таинственном мире, он не понимал, но знал, что все, что там делалось, было прекрасно, и был влюблен именно в эту таинственность совершавшегося."
test_text = clean_text(test_text)
test_text

'в это время левин часто бывал в доме щербацких и влюбился в дом щербацких как это ни странно может показаться но константин левин был влюблен именно в дом в семью в особенности в женскую половину семьи щербацких сам левин не помнил своей матери и единственная сестра его была старше его так что в доме щербацких он в первый раз увидал ту самую среду старого дворянского образованного и честного семейства которой он был лишен смертью отца и матери все члены этой семьи в особенности женская половина представлялись ему покрытыми какоюто таинственною поэтическою завесой и он не только не видел в них никаких недостатков но под этой поэтическою покрывавшею их завесой предполагал самые возвышенные чувства и всевозможные совершенства для чего этим трем барышням и нужно было говорить через день пофранцузски и поанглийски для чего они в известные часы играли попеременкам на фортепиано звуки которого всегда слышались у брата наверху где занимались студенты для чего ездили эти учителя французской ли

In [125]:
# Закодируем текст
def apply_encoding(test_text, enc_mapping):
    encoded_text = ''
    for letter in test_text:
        encoded_text += enc_mapping[letter]
    return encoded_text
encoded_text = apply_encoding(test_text, enc_mapping)

encoded_text

'цвмёхвцфалгвъацнквз дёхвуоц ъвцвжхлавсафу ьенйвнвцъщунъдгвцвжхлвсафу ьенйве евмёхвкнвдёф ккхвлхяаёвыхе т ёбдгвкхвехкдё кёнквъацнквуоъвцъщуъаквнлаккхвцвжхлвцвдалбщвцвхдхуаккхдёнвцвяакдеэщвыхъхцнкэвдалбнвсафу ьенйвд лвъацнквкавыхлкнъвдцхапвл ёафнвнважнкдёцакк гвдадёф ваюхвуоъ вдё фчаваюхвё евзёхвцвжхлавсафу ьенйвхквцвыафцопвф твэцнж ъвёэвд лэщвдфажэвдё фхюхвжцхфгкдехюхвхуф тхц ккхюхвнвзадёкхюхвдалапдёц вехёхфхпвхквуоъвънчаквдлафёбщвхёь внвл ёафнвцдавзъаковмёхпвдалбнвцвхдхуаккхдёнвяакде гвыхъхцнк выфаждё цъгъндбвалэвыхефоёолнве ехщёхвё нкдёцаккхщвыхмёнзадехщвт цадхпвнвхквкавёхъбехвкавцнжаъвцвкнйвкне енйвкажхдё ёехцвкхвыхжвмёхпвыхмёнзадехщвыхефоц цчащвнйвт цадхпвыфажыхъ ю ъвд лоавцхтцочаккоавзэцдёц внвцдацхтлхякоавдхцафчакдёц вжъгвзаюхвмёнлвёфалву фочкглвнвкэякхвуоъхвюхцхфнёбвзафатвжакбвыхиф кьэтденвнвых кюънпденвжъгвзаюхвхкнвцвнтцадёкоавз довнюф ънвыхыафалаке лвк вихфёаын кхвтцэенвехёхфхюхвцдаюж вдъоч ъндбвэвуф ё вк цафйэвюжавт кнл ъндбвдёэжакёовжъгвзаюхватжнънвмёнвэзнёаъгвиф кьэтдехпвън

In [15]:
def apply_decoding(encoded_text, freq_ngrams):
    char_cnt_encoded = dict(Counter(encoded_text).most_common())
    dec_mapping = dict(zip(dict(char_cnt_encoded).keys(), freq_ngrams.keys()))
    decoded_text = "".join([dec_mapping.get(char, '*') for char in encoded_text])
    return decoded_text
decoded_text = apply_decoding(encoded_text,freq_ngrams)
decoded_text

'т юсо тредб ветиа ынлсо ьптнв т моде щерьнцкиж и твшьивлб т мод щерьнцкиж кнк юсо аи лсрнаао доэес гокнзнсчлб ао коалснасиа ветиа ьпв твшьвеа идеаао т мод т ледчш т олоьеааолси т эеалкуш говотиау ледчи щерьнцкиж лнд ветиа ае годаив лтоей днсери и емиалстеаанб лелсрн еяо ьпвн лснрхе еяо снк ысо т моде щерьнцкиж оа т гертпй рнз утимнв су лндуш лрему лснрояо мторбалкояо оьрнзотнааояо и ыелсаояо ледейлстн косорой оа ьпв вихеа лдерсчш осцн и днсери тле ывеап юсой ледчи т олоьеааолси эеалкнб говотиан гремлснтвбвилч еду гокрпспди кнкошсо сниалстеааош гоюсиыелкош знтелой и оа ае совчко ае тимев т аиж аикнкиж аемолснскот ао гом юсой гоюсиыелкош гокрптнтхеш иж знтелой гремговнянв лндпе тозтпхеаапе ыутлстн и тлетоздоэапе лотерхеалстн мвб ыеяо юсид сред ьнрпхабд и ауэао ьпво яоторисч ыерез меач гофрнацузлки и гонаявийлки мвб ыеяо оаи т изтелсапе ынлп иярнви гогередеакнд ан форсегинао зтуки косорояо тлеямн лвпхнвилч у ьрнсн антержу яме знаиднвилч лсумеасп мвб ыеяо езмиви юси уыисевб фрнацузлкой ви

In [16]:
# Посмотрим на качество расшифровки
def accuracy(test_text, decoded_text):
    true_letters = sum([1 if c1==c2 else 0 for c1, c2 in zip(test_text, decoded_text)])
    return true_letters / len(test_text)

accuracy(test_text, decoded_text)

0.49586776859504134

Качество оставляет желать лучшего, слова не читаемы.

### 2 часть

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
проведите тестирование аналогично п.1, но при помощи биграмм.


In [68]:
# Подсчитаем частотность биграмм в тексте
def get_freq_bigrams(text, n=2):
    c = Counter(ngrams(text, n=n)).most_common()
    freq_ngrams = {k[0]+k[1]: v for k, v in dict(c).items()}
    return freq_ngrams

freq_2grams = get_freq_bigrams(text, n=2)


In [18]:
# Создадим маппинг для кодирования текста
enc_mapping = encode_mapping(freq_2grams)
enc_mapping

{'о ': 'нф',
 'и ': 'д ',
 'а ': 'вы',
 'е ': 'оа',
 ' с': ' м',
 ' н': 'пб',
 ' в': 'цв',
 ' п': 'шк',
 'то': ' ц',
 ' о': 'иж',
 'я ': 'аг',
 ' и': 'тз',
 'на': 'к ',
 'ст': 'юэ',
 'ь ': 'ео',
 'не': 'хв',
 'ал': 'чэ',
 ' к': 'нк',
 'но': 'гз',
 'го': 'чр',
 'по': 'цк',
 'ко': 'фн',
 'он': ' ъ',
 'ка': 'ан',
 'ов': 'ыл',
 ' т': 'га',
 'ни': 'ип',
 'л ': 'нь',
 '  ': 'оо',
 'ла': 'мв',
 'ра': 'еэ',
 'й ': 'эц',
 'ен': 'мг',
 ' д': 'дш',
 'во': 'ож',
 'м ': 'ог',
 'ро': 'нг',
 ' б': 'йм',
 'от': 'та',
 'у ': 'ва',
 'пр': 'мб',
 'ос': 'л ',
 ' ч': 'ци',
 'ло': 'цц',
 'ол': 'са',
 'ть': 'жо',
 'ел': 'ыу',
 'ер': 'ио',
 'ор': 'эф',
 'в ': 'чт',
 ' м': 'оч',
 'ли': 'фи',
 'ва': 'жд',
 'ре': 'цо',
 'за': 'юв',
 'н ': 'лз',
 'та': ' ё',
 'ск': ' ь',
 'те': 'яс',
 'ом': ' в',
 ' е': 'ью',
 'ле': 'ей',
 'ан': 'ац',
 'к ': 'лг',
 'ог': 'рв',
 'ри': 'ге',
 'ил': 'шш',
 'де': 'зч',
 'т ': 'ян',
 'ат': 'уу',
 'ет': 'еа',
 'од': 'нс',
 'ве': 'фч',
 'ак': 'а ',
 'ы ': 'шв',
 'ви': ' щ',
 'да': 'кш',

In [25]:
encoded_text = ''
for i in range(0, len(test_text), 2):
    encoded_text += enc_mapping.get(test_text[i:i+2], '|')
encoded_text

'чтяэнфхсзсагей щлзиеюэнфжсждньчтащтыхоиошцрьящтзцвблредвагчтащогякычос хрпанлгяэнфип мяхацгзочйюеашкз хлуужцаггзнк ъюэацьалзей щлзжсньчзеуейлзш мггзцвдш вцв мзсбвцвижох кюбл ьацвнемг ьцтшксаыл аваюпййд якычос хрпртогей щлзхвшк випньнакеэц шясгетзьюсюсыпкмгк агюпюэеэьючрймксвыюэчупгьючргаа ци ццвдш воаякычос хрп ъцвшкиог эцеэшийя гчэгавартжэффчьс ваюэчурвнфицэфря ьрвнфгоеэ бждюбрвнфд ётюэгзчр мзсошюэжднктаэфюмижлзжсньфипглзю иожоффтаклтзочууиод рбоажкмгшвяэюм мзсфуцвижох кюбл ьанемг ьюсшксаыл авымбс юэсцивфиимьюжэшкз мс жзмнка ки цгагюсыпкмггзффцкяэдачмфнффювфчохэцд  ъпбоа цигфнпбоа щзчньчтипрпипан хрпхващюэууфнчтгзшкнсйч цэццкяэдачмфнффцккфоисцпгффящелсцчмюмшкцохясазйчэ мфемжцвжбг пгюбмжцийяюэждтзцвюпожмийюмпоаохфчрамгюэжддшивциеянфяэш гацоогшцмсвсюштзпбтнгзймкснфчрожгежоцииойддшмгеоцкцчацысйи хтзшккоэзфихъ хдшивциеянф ъд чтьбфчюэмпоаиехэтзмзчэд цкй цотыеефепбвытчджжнтхгзелла хнктаэфрвнфрбеякш мфламфиимывймеэ ёпбсциойтоязчелацш чэнбеоюэзлмг ждшивциеянфйдсюфийчьаывкуясивя еэлызь ьюмтса

In [28]:
test_freq_2grams = get_freq_ngrams(encoded_text, n=2)
dec_mapping = dict(zip(dict(test_freq_2grams).keys(), freq_2grams.keys()))
decoded_text = ''

for i in range(0, len(encoded_text), 2):
    decoded_text += dec_mapping.get(encoded_text[i:i+2], '|')
decoded_text

'е рои ибк алдоегл ть си елероре евлььякопефиад оо илохдяале евв обеезннакаваьюрои  бя лыов нт чешнь д яжотойал ну ет соводл доегл елорафямдол пр и но то ао я к ныо сьреич  лаодо лю изацоь нисквионазфр кобеезннакамев доегл шиь  а борчтужтеечдног о чглочкр иавалаз сос ч в етало с убы ч внораа  по то агообеезннакаето ь коы теосимгржа тноонме рй халион с укии жижеьсзакии июостсер  кии  квн с н вя к ша серу рижеенсьл елорстбыл влкоакй ризе от отко кмаговя ичироеня к лло сьреич  лаодлю иза зь нисквилодвли с демстош ч рь д мнымулу раеш пноомочкр и нй ворооюнум й ц порете кетнего псём негоеганоре  бка бванакашиев сотм е  нь биру птеворооюнум й воашча дбый адлу днуень  грмнинь тя инвыо збы бы  выа гр сер оо азмуыйчеылгорепоху и сертоема еси ропрно гв пемндьощ онеыт н етаи  вмуогака коннто итивоьковфемсна оь ячязсттднатоема еси ет ке ыппо сылготьач ося т квоас гльякиннелоездшъяьб нлусонау рижекии маесещя иаьнстошаб еосивне дкооцттанлуовпр тиети сьт иымтоема еси ннглструодабхлднемятосф врзаеннрщ

In [30]:
test_text

'в это время левин часто бывал в доме щербацких и влюбился в дом щербацких как это ни странно может показаться но константин левин был влюблен именно в дом в семью в особенности в женскую половину семьи щербацких сам левин не помнил своей матери и единственная сестра его была старше его так что в доме щербацких он в первый раз увидал ту самую среду старого дворянского образованного и честного семейства которой он был лишен смертью отца и матери все члены этой семьи в особенности женская половина представлялись ему покрытыми какоюто таинственною поэтическою завесой и он не только не видел в них никаких недостатков но под этой поэтическою покрывавшею их завесой предполагал самые возвышенные чувства и всевозможные совершенства для чего этим трем барышням и нужно было говорить через день пофранцузски и поанглийски для чего они в известные часы играли попеременкам на фортепиано звуки которого всегда слышались у брата наверху где занимались студенты для чего ездили эти учителя французской ли

In [29]:
accuracy(test_text, decoded_text)

0.06675143038779402

Чтото совсем не очень получилось))

### 3 часть

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
реализуйте и протестируйте его, убедитесь, что результаты улучшились.

Используем МСМС-сэмлирование путем перестановки случайных букв в шифре и подсчетов количества биграмм в расшифрованном тексте, совпадающих с биграммами исходного текста.

In [157]:
def get_l_score(decoded_text, freq_2grams):
    # Cчитаем скор по произведению вероятностей биграмм
    test_freq_2grams = get_freq_bigrams(decoded_text, n=2)
    score = 0
    for bigram in test_freq_2grams:
        score += np.log(freq_2grams.get(bigram, 1 / len(test_freq_2grams)))
    return score

def get_new_mapping(mapping):
    # Получаем новый маппинг путем случайной перестановки букв
    new_mapping = mapping.copy()
    char1, char2 = random.sample(new_mapping.keys(), k=2)
    new_mapping[char1], new_mapping[char2] = new_mapping[char2], new_mapping[char1]
    return new_mapping

def decode_text(encoded_text, mapping):
    return "".join([mapping.get(char, '|') for char in encoded_text])

def metropolis_hastings_log_accept(l, l_new):
    if l_new > l:
        return True
    else:
        return (np.random.rand() < (np.exp(l_new-l)))
    
    
def mcmc(text, encoded_text, n_iters):

    # Считаем биграмы для подсчета скора
    freq2grams = get_freq_bigrams(text, n=2)
    # Делаем маппинг как в 1 части
    freq_train = get_freq_ngrams(text, n=1).keys()
    freq_test = get_freq_ngrams(encoded_text, n=1).keys()
    dec_mapping = dict(zip(freq_test, freq_train))
    # Декодируем текст
    decoded_text = decode_text(encoded_text, dec_mapping)
    # Считаем начальный скор
    l_score = get_l_score(decoded_text, freq_2grams)

    for i in tqdm.tqdm(range(n_iters)):
        # Получаем новый маппинг
        new_mapping = get_new_mapping(dec_mapping)
        # Декодируем текст
        new_decoded_text = decode_text(encoded_text, new_mapping)
        # Считаем начальный скор
        new_l_score = get_l_score(new_decoded_text, freq_2grams)

        if metropolis_hastings_log_accept(l_score, new_l_score):
            l_score = new_l_score
            dec_mapping = new_mapping

        
    print('Iteration:', i, 'l score:', l_score)
    print(decode_text(encoded_text, dec_mapping)[:200])
        
    return dec_mapping

n_iters = 5000
mcmc_mapping = mcmc(text, encoded_text, n_iters)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 724.53it/s]

Iteration: 4999 l score: 2477.512046262904
в это время левин часто бывал в доме щербацких и влюбился в дом щербацких как это ни странно может показаться но константин левин был влюблен именно в дом в семью в особенности в женскую половину семь


In [158]:
accuracy(test_text, decode_text(encoded_text, mcmc_mapping))

1.0

Результат просто отвал бошки) Даже за 5000 итераций получаются потрясающие результаты.

### 4 часть 
Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [161]:
new_encoded_text = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏' 
mcmc_mapping = mcmc(text, new_encoded_text, n_iters=100000)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:35<00:00, 2843.08it/s]

Iteration: 99999 l score: 1125.1593862175348
если вы вимите норхальный или подти норхальный текст у чтого соожбения который легко продитать скорее всего вы все смелали правильно и полудите хаксихальный жалл ша послемнее детвертое шамание курса з


Для хорошей расшифровки текста потребовалось больше итераций, но все равно алгоритм отрабатывает очень быстро.